# Fabric GSM: All-In-One Colab Notebook
Train GSM regression (100 epochs) and visualize weave patterns, warp/weft densities, and texture features (GLCM, LBP, Gabor). GPU-accelerated feature extraction with MobileNetV2. Designed for Google Colab T4 GPU.


In [ ]:
# 1) Colab Environment Setup (Pip + GPU)
import sys, os
if 'google.colab' in sys.modules:
    !pip -q install tensorflow==2.13.0 keras==2.13.0 scikit-learn scikit-image opencv-python-headless pyyaml openpyxl pandas numpy matplotlib pillow
    print('Installed dependencies for Colab.')
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print('GPUs available:', gpus)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
# 2) Reproducibility and Imports
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
import cv2, skimage, sklearn
print({'tensorflow': tf.__version__, 'numpy': np.__version__, 'pandas': pd.__version__, 'opencv': cv2.__version__, 'skimage': skimage.__version__, 'sklearn': sklearn.__version__})

In [ ]:
# 3) Colab Drive Mount + Paths
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    project_root = Path('/content/drive/MyDrive/GSM_fabric/fabric_gsm_pipeline')
else:
    project_root = Path('.')
print('Project root:', project_root)
dataset_root = project_root / 'data' / 'FabricNet'
excel_path = dataset_root / 'FabricNet_parameters.xlsx'
assert dataset_root.exists(), 'Dataset folder not found'
assert excel_path.exists(), 'Excel file not found'
import sys as _sys
_sys.path.insert(0, str(project_root))
from src.utils.config import load_config
config = load_config(project_root / 'configs' / 'config.yaml')

In [ ]:
# 4) Load Dataset via FabricNetDataset
from src.data.fabricnet_loader import FabricNetDataset
fabricnet = FabricNetDataset(dataset_root=dataset_root)
samples = fabricnet.get_all_samples()
df = pd.DataFrame([{
    'image_path': str(p),
    'specific_weight': gsm,
    'image_id': meta['image_id'],
    'warp': meta.get('warp'),
    'weft': meta.get('weft'),
    'texture': meta.get('texture'),
} for p, gsm, meta in samples])
print('Loaded samples:', len(df))
df.head()

In [ ]:
# 5) Visual Grid of 12 Samples
import math
n_show = 12
sel = df.sample(n=n_show, random_state=SEED)
cols = 4
rows = math.ceil(n_show/cols)
plt.figure(figsize=(12,9))
for i, r in enumerate(sel.itertuples(), 1):
    img = cv2.imread(r.image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(rows, cols, i)
    plt.imshow(img); plt.axis('off')
    plt.title(f"ID {r.image_id} GSM {r.specific_weight:.1f}")
plt.tight_layout(); plt.show()

In [ ]:
# 6) Stratified Train/Val/Test Split (qcut by GSM)
from sklearn.model_selection import StratifiedShuffleSplit
n_bins = 5
df['gsm_bin'] = pd.qcut(df['specific_weight'], q=n_bins, labels=False, duplicates='drop')
splitter = StratifiedShuffleSplit(n_splits=1, test_size=config.get('data', {}).get('test_ratio', 0.15), random_state=SEED)
train_idx, test_idx = next(splitter.split(df, df['gsm_bin']))
df_train = df.iloc[train_idx].reset_index(drop=True)
df_temp = df.iloc[test_idx].reset_index(drop=True)
val_ratio = config.get('data', {}).get('val_ratio', 0.15)
test_ratio = config.get('data', {}).get('test_ratio', 0.15)
temp_ratio = val_ratio + test_ratio
val_size = val_ratio / temp_ratio
splitter2 = StratifiedShuffleSplit(n_splits=1, test_size=1 - val_size, random_state=SEED)
val_idx, test_idx2 = next(splitter2.split(df_temp, df_temp['gsm_bin']))
df_val = df_temp.iloc[val_idx].reset_index(drop=True)
df_test = df_temp.iloc[test_idx2].reset_index(drop=True)
print(f"Split sizes -> train: {len(df_train)}, val: {len(df_val)}, test: {len(df_test)}")

In [ ]:
# 7) Preprocessing + Feature Extraction (Texture + Deep)
from src.preprocessing.image_preprocessor import ImagePreprocessor
from src.features.texture_features import TextureFeatureExtractor
from src.features.deep_features import DeepFeatureExtractor

pre_cfg = config.get('preprocessing', {})
preprocessor = ImagePreprocessor(
    target_size=tuple(config.get('data', {}).get('image_size', [224, 224])),
    enable_clahe=pre_cfg.get('enable_clahe', True),
    clahe_clip_limit=pre_cfg.get('clahe_clip_limit', 2.0),
    clahe_tile_size=tuple(pre_cfg.get('clahe_tile_size', [8, 8])),
    normalization_mode=pre_cfg.get('normalization', 'minmax'),
)

tex_cfg = config.get('texture_features', {})
texture_extractor = TextureFeatureExtractor(
    glcm_enabled=tex_cfg.get('glcm', {}).get('enable', True),
    glcm_distances=tex_cfg.get('glcm', {}).get('distances', [1,3]),
    glcm_angles=tex_cfg.get('glcm', {}).get('angles', [0,45,90,135]),
    glcm_levels=tex_cfg.get('glcm', {}).get('levels', 256),
    glcm_metrics=tex_cfg.get('glcm', {}).get('metrics', ['contrast','homogeneity','energy','correlation']),
    lbp_enabled=tex_cfg.get('lbp', {}).get('enable', True),
    lbp_radius=tex_cfg.get('lbp', {}).get('radius', 3),
    lbp_n_points=tex_cfg.get('lbp', {}).get('n_points', 8),
    lbp_method=tex_cfg.get('lbp', {}).get('method', 'uniform'),
    lbp_n_bins=tex_cfg.get('lbp', {}).get('n_bins', 59),
)

deep_cfg = config.get('deep_features', {})
deep_extractor = DeepFeatureExtractor(
    model_type=deep_cfg.get('model_type', 'MobileNetV2'),
    input_shape=(224,224,3),
    weights=deep_cfg.get('weights', 'imagenet'),
    pooling=deep_cfg.get('pooling', 'global_average'),
    preprocessing_mode=deep_cfg.get('preprocessing_mode', 'tf'),
    use_gpu=True,
)

def compute_features(df_split):
    tex_list, deep_list, y_list, id_list = [], [], [], []
    for row in df_split.itertuples():
        img_path = Path(row.image_path)
        img = preprocessor.preprocess(img_path)
        tex_feats = texture_extractor.extract_features(img)
        deep_feats = deep_extractor.extract_features(img)
        tex_list.append(tex_feats)
        deep_list.append(deep_feats)
        y_list.append(float(row.specific_weight))
        id_list.append(int(row.image_id))
    return (np.vstack(tex_list), np.vstack(deep_list), np.array(y_list, dtype=np.float32), np.array(id_list, dtype=np.int32))

tex_train, deep_train, y_train, id_train = compute_features(df_train)
tex_val, deep_val, y_val, id_val = compute_features(df_val)
tex_test, deep_test, y_test, id_test = compute_features(df_test)
print('Texture shapes:', tex_train.shape, tex_val.shape, tex_test.shape)
print('Deep shapes:', deep_train.shape, deep_val.shape, deep_test.shape)

In [ ]:
# 8) Feature Fusion + Standardization
from src.features.feature_fusion import FeatureFusion
X_train_raw = np.concatenate([tex_train, deep_train], axis=1)
X_val_raw = np.concatenate([tex_val, deep_val], axis=1)
X_test_raw = np.concatenate([tex_test, deep_test], axis=1)
feature_dim = X_train_raw.shape[1]
fusion_cfg = config.get('feature_fusion', {})
fusion = FeatureFusion(
    scaler_type=fusion_cfg.get('scaler_type', 'standard'),
    save_scaler=True,
    scaler_path=project_root / fusion_cfg.get('scaler_path', 'models/feature_scaler.pkl'),
)
X_train = fusion.fit_transform(X_train_raw)
X_val = fusion.transform(X_val_raw)
X_test = fusion.transform(X_test_raw)
print('Fused feature dim:', feature_dim)

In [ ]:
# 9) Keras MLP Regressor (100 Epochs)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

mlp = Sequential([
    Dense(256, activation='relu', input_shape=(feature_dim,)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='linear'),
])
mlp.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
EPOCHS = 100
BATCH_SIZE = int(config.get('training', {}).get('batch_size', 32))
history = mlp.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
)

In [ ]:
# 10) Evaluation and Save Predictions/Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
y_pred = mlp.predict(X_test).reshape(-1)
mae = mean_absolute_error(y_test, y_pred)
rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
r2 = float(r2_score(y_test, y_pred))
print({'mae': float(mae), 'rmse': rmse, 'r2': r2})
pred_path = project_root / config.get('evaluation', {}).get('predictions_path', 'results/predictions.csv')
pred_path.parent.mkdir(parents=True, exist_ok=True)
pd.DataFrame({'image_id': id_test, 'true_gsm': y_test, 'pred_gsm': y_pred}).to_csv(pred_path, index=False)
print('Saved predictions to', pred_path)
model_path = project_root / config.get('paths', {}).get('model_save_path', 'models/fabric_gsm_regressor.h5')
model_path.parent.mkdir(parents=True, exist_ok=True)
mlp.save(model_path)
print('Saved model to', model_path)
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True GSM'); plt.ylabel('Predicted GSM')
plt.title(f'Test R2={r2:.3f}, MAE={mae:.2f}, RMSE={rmse:.2f}')
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
plt.plot(lims, lims, 'r--'); plt.show()

In [ ]:
# 11) Visual Feature Sections: Edges/Hough, Orientation, FFT, GLCM/LBP, Gabor, Structure Tensor
import numpy as np
def visualize_edges_hough(image_path):
    img_bgr = cv2.imread(str(image_path))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(gray_blur, 50, 150)
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=80, minLineLength=30, maxLineGap=10)
    overlay = img_rgb.copy()
    if lines is not None:
        for l in lines[:,0]:
            x1,y1,x2,y2 = l
            cv2.line(overlay, (x1,y1), (x2,y2), (255,0,0), 2)
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(img_rgb); plt.title('Original'); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(edges, cmap='gray'); plt.title('Canny Edges'); plt.axis('off')
    plt.subplot(1,3,3); plt.imshow(overlay); plt.title('Hough Lines Overlay'); plt.axis('off')
    plt.tight_layout(); plt.show()

def orientation_histogram(image_path, n_bins=36):
    img_bgr = cv2.imread(str(image_path))
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (224,224), interpolation=cv2.INTER_CUBIC)
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    magnitude = np.sqrt(gx*gx + gy*gy)
    angle = (np.arctan2(gy, gx) * 180.0/np.pi) % 180.0
    hist, bins = np.histogram(angle, bins=n_bins, range=(0,180), weights=magnitude)
    centers = 0.5*(bins[:-1]+bins[1:])
    plt.figure(figsize=(8,3))
    plt.bar(centers, hist, width=180/n_bins)
    plt.title('Orientation Histogram'); plt.xlabel('Angle (deg)'); plt.ylabel('Weighted count'); plt.show()
    peak_angles = centers[np.argsort(hist)[-3:][::-1]]
    return {'hist': hist, 'centers': centers, 'peak_angles_deg': peak_angles}

def dominant_frequency(signal):
    sig = signal.astype(np.float32); sig = sig - sig.mean()
    N = len(sig)
    fft = np.fft.rfft(sig)
    mag = np.abs(fft); mag[0] = 0
    peak_idx = np.argmax(mag)
    freq = peak_idx / N
    period_px = (1.0 / freq) if freq > 0 else np.inf
    return freq, period_px, mag

def estimate_warp_weft(image_path):
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.float32)/255.0
    row_mean = img.mean(axis=1)
    col_mean = img.mean(axis=0)
    warp_freq, warp_period, warp_mag = dominant_frequency(row_mean)
    weft_freq, weft_period, weft_mag = dominant_frequency(col_mean)
    fig, axs = plt.subplots(2,2, figsize=(10,6))
    axs[0,0].plot(row_mean); axs[0,0].set_title(f'Row Mean (Warp) period~{warp_period:.1f}px')
    axs[0,1].plot(warp_mag); axs[0,1].set_title(f'Warp FFT mag (peak {warp_freq:.4f})')
    axs[1,0].plot(col_mean); axs[1,0].set_title(f'Col Mean (Weft) period~{weft_period:.1f}px')
    axs[1,1].plot(weft_mag); axs[1,1].set_title(f'Weft FFT mag (peak {weft_freq:.4f})')
    plt.tight_layout(); plt.show()
    return {'warp_freq': warp_freq, 'warp_period_px': warp_period, 'weft_freq': weft_freq, 'weft_period_px': weft_period}

from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
def show_glcm_lbp(image_path, distances=[1,3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, lbp_radius=3):
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img_norm = (img.astype(np.float32)/255.0)
    quantized = (img_norm*(levels-1)).astype(np.uint8)
    glcm = graycomatrix(quantized, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
    metrics = {
        'contrast': graycoprops(glcm, 'contrast'),
        'homogeneity': graycoprops(glcm, 'homogeneity'),
        'energy': graycoprops(glcm, 'energy'),
        'correlation': graycoprops(glcm, 'correlation'),
    }
    lbp = local_binary_pattern(img_norm, P=lbp_radius*8, R=lbp_radius, method='uniform')
    n_bins = 59
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))
    hist = hist.astype(np.float32); hist = hist / (hist.sum() + 1e-8)
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(img, cmap='gray'); plt.title('Image'); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(glcm[:,:,0,0], cmap='magma'); plt.title('GLCM (d=1, θ=0)'); plt.colorbar(); plt.axis('off')
    plt.subplot(1,3,3); plt.bar(np.arange(n_bins), hist); plt.title('LBP Histogram')
    plt.tight_layout(); plt.show()
    return metrics, hist

from skimage.filters import gabor
def show_gabor_bank(image_path, frequencies=[0.1, 0.2, 0.3], thetas=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.float32)/255.0
    fig, axs = plt.subplots(len(frequencies), len(thetas), figsize=(12,8))
    for i,f in enumerate(frequencies):
        for j,t in enumerate(thetas):
            real, imag = gabor(img, frequency=f, theta=t)
            axs[i,j].imshow(real, cmap='gray')
            axs[i,j].set_title(f'f={f}, θ={int(t*180/np.pi)}°')
            axs[i,j].axis('off')
    plt.tight_layout(); plt.show()

from skimage.feature import structure_tensor, structure_tensor_eigvals
def show_orientation_field(image_path):
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.float32)/255.0
    Axx, Axy, Ayy = structure_tensor(img, sigma=1.0)
    e1, e2 = structure_tensor_eigvals(Axx, Axy, Ayy)
    orientation = 0.5*np.arctan2(2*Axy, Axx - Ayy)
    plt.figure(figsize=(6,5))
    plt.imshow(orientation, cmap='twilight'); plt.colorbar(); plt.title('Orientation Field'); plt.axis('off'); plt.show()

# Demo on one sample
sample_path = df_train.sample(1, random_state=SEED)['image_path'].iloc[0]
visualize_edges_hough(sample_path)
oh = orientation_histogram(sample_path)
print('Top orientation peaks (deg):', oh['peak_angles_deg'])
est = estimate_warp_weft(sample_path)
print('Warp/Weft estimation:', est)
_ = show_glcm_lbp(sample_path)
show_gabor_bank(sample_path)
show_orientation_field(sample_path)

In [ ]:
# 12) Dataset-wide Visual Feature Summary and Correlations
def summarize_features(df_split, limit=None):
    rows = []
    count = 0
    for r in df_split.itertuples():
        if limit and count >= limit:
            break
        p = r.image_path
        est = estimate_warp_weft(p)
        oh = orientation_histogram(p)
        metrics, hist = show_glcm_lbp(p)
        contrast_mean = float(metrics['contrast'].mean())
        homog_mean = float(metrics['homogeneity'].mean())
        energy_mean = float(metrics['energy'].mean())
        corr_mean = float(metrics['correlation'].mean())
        lbp_entropy = float(-(hist*np.log(hist+1e-8)).sum())
        rows.append({
            'image_id': r.image_id,
            'gsm': float(r.specific_weight),
            'warp_freq': est['warp_freq'],
            'weft_freq': est['weft_freq'],
            'warp_period_px': est['warp_period_px'],
            'weft_period_px': est['weft_period_px'],
            'orientation_peak_deg_1': float(oh['peak_angles_deg'][0]),
            'orientation_peak_deg_2': float(oh['peak_angles_deg'][1]),
            'orientation_peak_deg_3': float(oh['peak_angles_deg'][2]),
            'glcm_contrast_mean': contrast_mean,
            'glcm_homogeneity_mean': homog_mean,
            'glcm_energy_mean': energy_mean,
            'glcm_correlation_mean': corr_mean,
            'lbp_entropy': lbp_entropy,
        })
        count += 1
    return pd.DataFrame(rows)

summary_df = summarize_features(df, limit=130)
out_path = project_root / 'results' / 'features_visual_summary.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)
summary_df.to_csv(out_path, index=False)
print('Saved feature summary to', out_path)
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.scatter(summary_df['warp_freq'], summary_df['gsm']); plt.xlabel('Warp freq'); plt.ylabel('GSM')
plt.subplot(1,2,2); plt.scatter(summary_df['weft_freq'], summary_df['gsm']); plt.xlabel('Weft freq'); plt.ylabel('GSM')
plt.tight_layout(); plt.show()
print('Spearman correlations:')
print(summary_df[['warp_freq','weft_freq','warp_period_px','weft_period_px','glcm_contrast_mean','glcm_homogeneity_mean','lbp_entropy','gsm']].corr(method='spearman'))